<h1>The apartment finder:</h1>

<h2>First, we ge the apartment information from our Kijiji scraper:</h2>

In [4]:
                                                                                                                                                                                                    #Get informations from Kijiji scraper:
%run kijiji_scraper.ipynb

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_linux64.zip


[WDM] - Driver has been saved in cache [/home/kurosuzume/.wdm/drivers/chromedriver/linux64/85.0.4183.87]


There are no more apartments with those filters right now. 
chrome closed


In [31]:
import html
from IPython.display import HTML


<h2>The apartment addresses are shown below.</h2>

In [32]:
text_address

['171 Priestman St, Fredericton, NB E3B 3B3, Canada',
 '35 Patience Ln, Fredericton, NB E3B 0K4, Canada',
 '382 George Street, E3B 1J7, Fredericton, NB',
 '382 George Street, Fredericton E3B 1J7 NB',
 '78 York Street, E3B3N5, Fredericton, NB',
 '641 Canterbury Dr, Fredericton, NB E3B 4M6, Canada',
 '15 Waggoners Lane, Fredericton, NB, E3B 2L2',
 ', Fredericton E3A3R2 NB, Canada',
 '706 Valleyview St, Fredericton, NB E3B 4E8, Canada',
 'Valleyview St, Fredericton, NB E3B 4E8, Canada',
 '136 Springwater Ln, New Maryland, NB E3C 1J7, Canada',
 ', Fredericton E3B 4S7 NB, Canada',
 '527 PRIESTMAN ST, FREDERICTON, NB, E3B 3B6',
 '202 Parkside Dr, Fredericton, NB E3B 5V7, Canada',
 '171 Priestman St, Fredericton, NB E3B 3B3, Canada',
 '74 Kilburn St, Fredericton, NB E3A 1Y1, Canada',
 'York, Fredericton E3B9K6 NB, Canada',
 '260 WETMORE RD, FREDERICTON, NB, E3B 5V4',
 'E3B5L9, Canada',
 '176 Saint John Street, E3B 4A9, Fredericton, NB',
 '151 Queen Street, Fredericton, NB, E3B 7J2',
 '677 Win

In [33]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h2>Now, we import all libraries needed:</h2>

In [38]:
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files

!pip install pgeocode
import pgeocode

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't competed 
from geopy.geocoders import Nominatim

#Matplotlib and associated ploting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium-0.5.0 --yes
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from geopy import distance  
print("Libraries imported successfully")



Solving environment: done

# All requested packages already installed.

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - folium-0.5.0

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Libraries imported successfully


<h2>The reference point is the NBCC campus in New Brunswick.</h2>
<h3>We set the location coordinates and the radius we will use to find the apartments and amenities:</h3>

In [39]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000


The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


In [40]:
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(AD_title))

<h3>Ensuring all te data is in the right format:</h3>

In [41]:
#Before finding the geo-locations of the addresses given, we need to verify if they are in the right format:
main_address = []
unaccepted_address_index = []

for address in text_address:
    house = address.split(',')
    house = house[0]+",Fredericton, CA"
    main_address.append(house)

for i in range(len(main_address)):
    if main_address is None:    
        main_address.remove(main_address[i])
        ap_prices.remove(ap_prices[i])
        title_pic_change.remove(title_pic_change[i])
        num_bedrooms.remove(num_bedrooms[i])
        num_bathrooms.remove(num_bathrooms[i])
#print (main_address)
#print(len(main_address))
#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))

In [42]:
geo_location = main_address

In [43]:
#Now, to find the geo location of each of the addresses:

index_to_remove = []
geo_location = []
for address in range(len(main_address)):
    appended_location = geolocator.geocode(main_address[address])
    geo_location.append(appended_location)
    if appended_location is None:
        index_to_remove.append(address)
index_to_remove = index_to_remove.reverse()

if index_to_remove is None:
        pass
else: 
    for i in index_to_remove:

        geo_location.remove(geo_location[address])
        ap_prices.remove(ap_prices[address])
        num_bedrooms.remove(num_bedrooms[address])
        num_bathrooms.remove(num_bathrooms[address])          

#print(len(ap_prices))
#print(len(num_bedrooms))
#print(len(num_bathrooms))
#print(len(geo_location))


<h3>Showing the Location information for all the apartments</h3>

In [44]:
geo_location

[Location(171, Priestman Street, Rabbit Town, Uptown, Fredericton, York County, New Brunswick, E3B 3H2, Canada, (45.94504295, -66.6653624, 0.0)),
 None,
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(George Street, Carleton, Downtown, Fredericton, York County, New Brunswick, E3B 3L7, Canada, (45.9596959, -66.6444762, 0.0)),
 Location(York Street, Rabbit Town, Downtown, Fredericton, York County, New Brunswick, E3B 0V4, Canada, (45.9546808, -66.6503261, 0.0)),
 Location(Canterbury Drive, Dunn's Crossing, Skyline Acres, Fredericton, York County, New Brunswick, E3B 4K9, Canada, (45.936282, -66.634257, 0.0)),
 Location(15, Waggoners Lane, Rabbit Town, Sunshine Gardens, Fredericton, York County, New Brunswick, E3B 2L4, Canada, (45.95844665, -66.66120270972982, 0.0)),
 Location(Fredericton, York County, New Brunswick, Canada, (45.966425, -66.645813, 0.0)),
 Location(706, Valleyview Street, Colle

In [45]:
location_info = []
discarded_index = []
for location in range(len(geo_location)):
    if not geo_location[location] == None: 
        location_info.append([geo_location[location].address, geo_location[location].latitude, geo_location[location].longitude ])
    else: 
        discarded_index.append(location)
    

In [46]:
#print(location_info)
#print(len(location_info))

<h3>Using the Introduction_to_the_Problem notebook to retrieve the map.</h3>

In [47]:
#Get the map code from the introduction notebook:
%run Introduction_to_the_Problem.ipynb

The college address is: Duffie Drive, College Hill, Skyline Acres, Fredericton, York County, New Brunswick, E3B 9W4, Canada


In [48]:
address = '"26 Duffie Dr, Fredericton, NB , Canada"'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the NBCC Fredericton campus are {}, {}.".format(latitude, longitude))
radius = 10000



The geographical coordinates of the NBCC Fredericton campus are 45.9428786, -66.6456084.


<h3>Foursquare credentials:</h3>
<p>Please, fill with your foursquare information:</p>

In [54]:
CLIENT_ID = 'GRUEM4VGSOO2HFMKCVFHP155RMR0VF4B0G1XZF3XSD5IIUQ4' # your Foursquare ID
CLIENT_SECRET = 'HQ25A2EDHJR5QLUH0DQSYHH2GMMDZMD5GIZ2055WPL35XQJ3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

<p>Since it wasn't possible to use iterative method to find the amenities, we will do it in a non pythonical way:</p>

In [55]:
#Now, to get the nearby venues according to the important amenities:
amenities_list = ["Grocery Store","Supermarket", "Gym", "Drugstore", "Hospital", "Candy Store"]
search_query0 = "Grocery Store"
search_query1 = "Supermarket"
search_query2 = "Gym"
search_query3 = "Drugstore"
search_query4 = "Hospital"
search_query5 = "Candy Store"


In [56]:
url0 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query0, radius, LIMIT)
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query4, radius, LIMIT)
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query5, radius, LIMIT)


In [57]:
results0 = requests.get(url0).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
results3 = requests.get(url3).json()
results4 = requests.get(url4).json()
results5 = requests.get(url5).json()



In [58]:
# assign relevant part of JSON to venues
venues0 = results0['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']
venues3 = results3['response']['venues']
venues4 = results4['response']['venues']
venues5 = results5['response']['venues']

# transform venues into dataframes
dataframe0 = json_normalize(venues0)
dataframe1 = json_normalize(venues1)
dataframe2 = json_normalize(venues2)
dataframe3 = json_normalize(venues3)
dataframe4 = json_normalize(venues4)
dataframe5 = json_normalize(venues5)


/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/home/kurosuzume/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use p

<h2>Results from foursquare will converge into one dataframe:</h2>

In [59]:
# now converge all results into one dataframe:

dataframe0 = pd.concat([dataframe0,dataframe1], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe2], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe3], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe4], axis=0, ignore_index=True)
dataframe0 = pd.concat([dataframe0,dataframe5], axis=0, ignore_index=True)



dataframe0

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c545e5db426ef3b11cc7e8a,Dick's Grocery Store,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",v-1600462427,False,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,NaN
1,4badfab8f964a520a3773be3,Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600462427,False,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
2,4c3c8e25282203bb55a3fcda,American Eagle & Aerie Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1600462427,False,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,NaN
3,4c1030d7ce57c928688981d2,UPS Store,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",v-1600462427,False,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,NaN
4,53f7c437498eacb60d222490,Great Canadian Dollar Store,"[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",v-1600462427,False,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,NaN
5,50a8f005e4b0e4f42e033a2a,Gap Factory Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1600462427,False,"11 Trinity Ave, Fredericton",45.930211,-66.662416,"[{'label': 'display', 'lat': 45.930211, 'lng':...",1918,E3C 0B9,CA,Fredericton,NB,Canada,"[11 Trinity Ave, Fredericton, Fredericton NB E...",NaN,NaN
6,4dbf1b1b6e810768bf9ccf5a,Raymonde's General Store,[],v-1600462427,False,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
7,5b93ade0f193c0002c473a51,The UPS Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600462427,False,527 Dundonald St,45.954230,-66.647487,"[{'label': 'display', 'lat': 45.95422952664039...",1271,E3B 1X5,CA,Fredericton,NB,Canada,"[527 Dundonald St, Fredericton NB E3B 1X5, Can...",NaN,NaN
8,4cd979481647a0936c21e74c,Everything For A Dollar Store,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1600462427,False,NaN,45.940661,-66.664114,"[{'label': 'display', 'lat': 45.94066140773827...",1453,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,NaN
9,4be89132d837c9b6447fa506,University Cafe & Market Convenience Store,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",v-1600462427,False,University Avenue,45.953861,-66.635277,"[{'label': 'display', 'lat': 45.95386136941374...",1460,NaN,CA,Fredericton,NB,Canada,"[University Avenue (Shore St), Fredericton NB,...",Shore St,NaN


In [60]:
dataframe0.shape

(94, 18)

<p>The dataframe is filtered:</p>

In [61]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe0.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe0.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Dick's Grocery Store,Smoke Shop,480 Westmorland St,45.941957,-66.663877,"[{'label': 'display', 'lat': 45.94195715102332...",1417,E3B 3M9,CA,Fredericton,NB,Canada,"[480 Westmorland St, Fredericton NB E3B 3M9, C...",NaN,4c545e5db426ef3b11cc7e8a
1,Dollar Store,Miscellaneous Shop,NaN,45.942453,-66.650432,"[{'label': 'display', 'lat': 45.94245279861792...",376,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4badfab8f964a520a3773be3
2,American Eagle & Aerie Store,Clothing Store,1381 Regent Street,45.935129,-66.664601,"[{'label': 'display', 'lat': 45.93512887044409...",1704,E3C 1A2,CA,Fredericton,NB,Canada,"[1381 Regent Street, Fredericton NB E3C 1A2, C...",NaN,4c3c8e25282203bb55a3fcda
3,UPS Store,Paper / Office Supplies Store,527 Dundonald St,45.953991,-66.647256,"[{'label': 'display', 'lat': 45.95399123193, '...",1243,E3B 1X6,CA,Fredericton,NB,Canada,"[527 Dundonald St (Beaverbrook ct), Fredericto...",Beaverbrook ct,4c1030d7ce57c928688981d2
4,Great Canadian Dollar Store,Discount Store,Uptown Center,45.939575,-66.663973,"[{'label': 'display', 'lat': 45.93957493230641...",1468,NaN,CA,Fredericton,NB,Canada,"[Uptown Center (Prospect St), Fredericton NB, ...",Prospect St,53f7c437498eacb60d222490
5,Gap Factory Store,Clothing Store,"11 Trinity Ave, Fredericton",45.930211,-66.662416,"[{'label': 'display', 'lat': 45.930211, 'lng':...",1918,E3C 0B9,CA,Fredericton,NB,Canada,"[11 Trinity Ave, Fredericton, Fredericton NB E...",NaN,50a8f005e4b0e4f42e033a2a
6,Raymonde's General Store,None,NaN,45.931152,-66.639836,"[{'label': 'display', 'lat': 45.93115187969736...",1379,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4dbf1b1b6e810768bf9ccf5a
7,The UPS Store,Miscellaneous Shop,527 Dundonald St,45.954230,-66.647487,"[{'label': 'display', 'lat': 45.95422952664039...",1271,E3B 1X5,CA,Fredericton,NB,Canada,"[527 Dundonald St, Fredericton NB E3B 1X5, Can...",NaN,5b93ade0f193c0002c473a51
8,Everything For A Dollar Store,Miscellaneous Shop,NaN,45.940661,-66.664114,"[{'label': 'display', 'lat': 45.94066140773827...",1453,NaN,CA,Fredericton,NB,Canada,"[Fredericton NB, Canada]",NaN,4cd979481647a0936c21e74c
9,University Cafe & Market Convenience Store,Candy Store,University Avenue,45.953861,-66.635277,"[{'label': 'display', 'lat': 45.95386136941374...",1460,NaN,CA,Fredericton,NB,Canada,"[University Avenue (Shore St), Fredericton NB,...",Shore St,4be89132d837c9b6447fa506


In [62]:
amenities= dataframe_filtered["name"].to_list()
amenities_categories = dataframe_filtered["categories"].to_list()
amenities_list_lat = dataframe_filtered["lat"].to_list()
amenities_list_lng = dataframe_filtered["lng"].to_list()


<h3>And finally, we add the amenities to our Folium map.</h3>
<p>The amenities will be displayed as a blue dot with brown outline. </p>

In [63]:

for i in range(len(geo_location)):
    if not geo_location[i] == None: 
        tooltip = "Apartment"
        folium.Marker([geo_location[i].latitude,geo_location[i].longitude], popup=AD_title[i], tooltip = tooltip ).add_to(map_Fred)
    else:
        pass

In [64]:

# add the Italian amenities as blue circle markers with red outline
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='brown',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip = label
    ).add_to(map_Fred)
map_Fred

<h3>Now, using geopy, we can verify amenities within a 800m radius from an apartment.</h3>

In [65]:
from geopy import point
#Function to find amenities in a 500m radius from a center point:
def find_in_radius(center_name, center_lat,center_lng, amenities_list_lat, amenities_list_lng, amenities_list, amenities_categories):
    """(float, float, list) -> list
        Retruns a list of amenities within a 800 radius from a center point.
    """
    center_lat = str(center_lat)
    center_lng = str(center_lng)
    center_point = point.Point(str(center_lat)+ "," + str(center_lng))
    
    within_radius = []
    amenities_cat = []
    for amenity in range(len(amenities_list)):
        amenities_list_point = point.Point(str(amenities_list_lat[amenity])+ "," + str(amenities_list_lng[amenity]))
        amen_dis = distance.distance(center_point, amenities_list_point).km
        if amen_dis <= 0.80: #1000 meters
            within_radius.append(amenities_list[amenity])
            amenities_cat.append(amenities_categories[amenity])
    return center_name,within_radius,amenities_cat

        

In [66]:
#num_bedrooms
#num_bathrooms
#appartment_price_accepted
for type in range(len(amenities_categories)):
    if amenities_categories[type] == None:
        amenities_categories[type] = "None"

In [67]:
full_list = []
for i in range(len(location_info)):
    full_list.append(find_in_radius(location_info[i][0],location_info[i][1], location_info[i][2], amenities_list_lat, amenities_list_lng, amenities,amenities_categories))

In [68]:
#full_list


In [69]:
reverse_discarded_index = discarded_index.reverse()

In [70]:
discarded_index

[37, 22, 18, 15, 10, 1]

In [71]:
for i in discarded_index:
    ap_prices.remove(ap_prices[i])
    num_bedrooms.remove(num_bedrooms[i])
    num_bathrooms.remove(num_bathrooms[i])

In [72]:
apt_address = []
amenity_name = []
amenity_type= []
for i in range(len(full_list)):
    apt_address.append(full_list[i][0])
    amenity_name.append(full_list[i][1])
    amenity_type.append(full_list[i][2])

In [73]:
separator = ","
prepared_amenity = []
for i in range(len(amenity_type)):
    sep = separator.join(amenity_type[i])
    prepared_amenity.append(sep)


In [75]:
for i in range(len(prepared_amenity)):
    prepared_amenity[i] = str(prepared_amenity[i]).split(',')
    for j in range(len(prepared_amenity[i])):
        prepared_amenity[i][j].split(",")

    

In [76]:
apartment_df= pd.DataFrame(list(zip(apt_address, amenity_name, prepared_amenity, ap_prices, num_bedrooms, num_bathrooms)))

In [77]:
len(prepared_amenity)
result = list(set(x for l in prepared_amenity for x in l))


In [78]:
apartment_df = pd.concat([apartment_df,pd.get_dummies(apartment_df[2].apply(pd.Series).stack()).sum(level=0)], axis = 1)

In [79]:
apartment_df = apartment_df.replace({5:{'Bathrooms: 1': 1, 'Bathrooms: 2': 2}})

apartment_df = apartment_df.replace({4:{'Bedrooms: Bachelor/Studio': 0, 'Bedrooms: 1': 1,'Bedrooms: 2': 2}})


In [80]:
apartment_df = apartment_df.rename(columns={0: "Apartment Address", 1: "Amenities",2:"Amenity type", 3:"Rent price", 4:"Bedrooms", 5:"Bathrooms"})


In [81]:
apartment_df

,Apartment Address,Amenities,Amenity type,Rent price,Bedrooms,Bathrooms,,Candy Store,Clothing Store,College Gym,College Rec Center,Convenience Store,Department Store,Discount Store,Electronics Store,Furniture / Home Store,General Travel,Grocery Store,Gym / Fitness Center,Hospital,IT Services,Miscellaneous Shop,Mobile Phone Shop,None,Nudist Beach,Paper / Office Supplies Store,Smoke Shop,Tourist Information Center,Veterinarian
0,"171, Priestman Street, Rabbit Town, Uptown, Fr...","[Dick's Grocery Store, Great Canadian Dollar S...","[Smoke Shop, Discount Store, Miscellaneous Sho...","$1,495.00",0,2,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,2,0,0,0,0,2,0,1
1,"George Street, Carleton, Downtown, Fredericton...","[UPS Store, The UPS Store, Kwik-Way Convenienc...","[Paper / Office Supplies Store, Miscellaneous ...",$550.00,0,1,0,0,2,0,0,2,0,0,0,2,2,0,1,0,0,2,0,0,1,2,0,2,1
2,"George Street, Carleton, Downtown, Fredericton...","[UPS Store, The UPS Store, Kwik-Way Convenienc...","[Paper / Office Supplies Store, Miscellaneous ...",$600.00,0,1,0,0,2,0,0,2,0,0,0,2,2,0,1,0,0,2,0,0,1,2,0,2,1
3,"York Street, Rabbit Town, Downtown, Fredericto...","[UPS Store, The UPS Store, Kwik-Way Convenienc...","[Paper / Office Supplies Store, Miscellaneous ...",$650.00,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,1
4,"Canterbury Drive, Dunn's Crossing, Skyline Acr...","[Raymonde's General Store, Raymonde's General ...","[None, None]",$740.00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0
5,"15, Waggoners Lane, Rabbit Town, Sunshine Gard...","[Kwik-Way Convenience Store, Kwik-Way Convenie...","[Convenience Store, Convenience Store]",$750.00,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"Fredericton, York County, New Brunswick, Canada","[Urban Almanac, The Travel Store, Militia Arms...","[Furniture / Home Store, General Travel, Touri...",$750.00,1,1,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,2,0
7,"706, Valleyview Street, College Hill, Skyline ...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[Miscellaneous Shop, Gym / Fitness Center, Col...",$750.00,1,1,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0
8,"Valleyview Street, College Hill, Skyline Acres...","[Dollar Store, Lady Beaverbrook Gym - UNB, Sou...","[Miscellaneous Shop, Gym / Fitness Center, Col...",$775.00,1,1,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0
9,"Fredericton, York County, New Brunswick, Canada","[Urban Almanac, The Travel Store, Militia Arms...","[Furniture / Home Store, General Travel, Touri...",$800.00,1,1,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,2,0


In [82]:
apartment_df.to_csv("apartment_dataframe.csv", index=True)
